# Gensim Doc2Vec

모든 문장의 vector를 구한 다음 가장 비슷한 vector를 가진 문장을 찾는다.

1. 전체 문장을 소문자화 한다.
2. 알파벳으로만 된 단어 개수가 5개 이상, 단어 개수가 15개 이하인 문장만 사용한다.

In [1]:
import boto3
import pickle
import os
import re
from nltk import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from tqdm import tqdm_notebook

model_file = 'doc2vec.model'
pickle_file = 'corpus.pickle'
learning_on = input('Learning on? (y/n)...').lower()

C:\Users\Bangjungyip\AppData\Local\conda\conda\envs\gensim\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Learning on? (y/n)...y


In [3]:
if not os.path.exists('txtData/'):
    os.mkdir('txtData')

pickle 파일이 존재하지 않으면 S3에서 문장 데이터를 다운받아 pickle 파일로 저장한다.

In [17]:
p = re.compile('[a-zA-Z]+')  # 알파벳으로만 된 단어 패턴

original_sents = []
common_texts = []

overwrite = 'n'
if os.path.exists(pickle_file):  # pickle 파일이 존재하면
    overwrite = input('Overwrite? (y/n)...').lower()

if overwrite == 'y' or not os.path.exists(pickle_file):
    bucket = boto3.resource('s3').Bucket('learningdatajchswm9')
    download_cnt = 0

    for i in tqdm_notebook(range(12876)):
        # local_file = 'sents/HOO' + str(i) + 'abstract.txt'
        # local_file = 'sents/HOO' + str(i) + 'content.txt'
        local_file = 'txtData/HOO' + str(i) + '.txt'
        
        try:
            if not os.path.exists(local_file):
                bucket.download_file(local_file, local_file)
                
            with open(local_file, 'r', encoding='UTF-8') as f:
                sentences = f.read().splitlines()
                original_sents += sentences
                for sent in sentences:
                    tokenized = word_tokenize(sent.lower())
                    common_texts.append(tokenized)
                        
            download_cnt += 1
            
        except Exception as e:
            # print(i, e)
            pass

    with open(pickle_file, 'wb') as f:
        pickle.dump(common_texts, f, pickle.HIGHEST_PROTOCOL)
    print('%d files downloaded. %d sentences saved.' % (download_cnt, len(common_texts)))
else:
    with open(pickle_file, 'rb') as f:
        common_texts = pickle.load(f)
        print(len(common_texts), 'sentences loaded.')

Overwrite? (y/n)...y


186 files downloaded. 179767 sentences saved.


In [18]:
for sent in original_sents[:10]:
    print(sent)

len(original_sents)

Identifying Student Difficulties with Basic Data Structures Daniel Zingaro University of Toronto Mississauga daniel.zingaro@utoronto.ca Michael Clancy University of California, Berkeley Cynthia Taylor Oberlin College ctaylor@oberlin.edu Cynthia Lee Stanford University Kevin C. Webb Swarthmore College Leo Porter University of California, San Diego leporter@eng.ucsd.edu Soohyun Nam Liao University of California, San Diego ABSTRACT To be effective instructors and CS education researchers, we must identify and understand student difficulties surrounding core computing topics.
This study examines student difficulties with the basic data structures commonly found in CS2 courses.
Initial exploration of student thinking began with think-aloud interviews with students.
These interviews centered on open-ended questions that were iteratively improved upon based on analysis of interview transcripts.
The revised open-ended questions were then posed to 249 students during an end-of-term final exam s

179767

모델을 학습시키거나 학습된 모델을 가져온다.

In [19]:
%%time

if learning_on == 'y':
    # Initialize & train a model
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
    model = Doc2Vec(documents, vector_size=300, window=7, min_count=10, workers=8, epochs=25, dm=1, dm_mean=1, dbow_words=1)

    model.save(model_file)
    
else:
    model = Doc2Vec.load(model_file)
    
model_parameters = 'vector_size=%d,window=%d,min_count=%d,epochs=%d' % (model.vector_size, model.window, model.vocabulary.min_count, model.epochs)

Wall time: 4min


In [20]:
model.delete_temporary_training_data()

In [23]:
%%time

input_sents = ['Software is becoming an increasingly large part in the automotive industry.',
               'Experimental results on real and synthetic datasets demonstrate the effectiveness of our model.',
              'This thesis introduces dynamic software updating.',
              'We also describe a variant that scales to high-dimensional domains.',
              'Cloud computing faces many problems.',
              'update',
              'We consider that we can face problems',
              'We faces many problems',
              'We are developing something good.']

for input_sent in input_sents:
    input_token = word_tokenize(input_sent.lower())
    
    average_n = 10
    vector = 0
    for k in range(average_n):
        vector += model.infer_vector(input_token)
    vector /= average_n

    # Search for the most similar sentences
    similar_sentences = model.docvecs.most_similar(positive=[vector], topn=10)
    
    # topn개의 문장 중에서 이상한 문장 지우기
    good_sents = []
    for sent_index in range(len(similar_sentences)):
        sent = original_sents[similar_sentences[sent_index][0]]
        is_good_sent = True
        # 문장의 단어 수가 5보다 작고
        if len(sent) < 5:
            is_good_sent = False
        # 한 단어의 길이가 20을 넘지 않고, 영어가 아닌 문장이 너무 많이 들어가 있지 않아야
        good_word_cnt = 0
        for word in sent:
            if len(word) >= 20:
                is_good_sent = False
                break
            if p.match(word):
                good_word_cnt += 1
        if good_word_cnt < 0.5 * len(sent):
            is_good_sent = False
            
        if is_good_sent:
            good_sents.append(similar_sentences[sent_index])

    show_cnt = 0  # 3개 문장만 보여줄거다.
    # Log
    log_path = 'logs/' + model_parameters + '.txt'
    with open(log_path, 'a+', encoding='UTF-8') as f:
        f.write(str(input_sent) + ' <- input_sent\n')
        for sent_num, similarity in good_sents:
            if similarity < 0.5:
                break
            f.write(str(original_sents[sent_num]) + ' ' + str(similarity) + '\n')
            show_cnt += 1
            if show_cnt >= 3:  # 3개 문장만 보여줄거다.
                break
        f.write('\n')

Wall time: 285 ms


In [22]:
model.most_similar(positive=['describe'])

C:\Users\Bangjungyip\AppData\Local\conda\conda\envs\gensim\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('define', 0.6457910537719727),
 ('explain', 0.6221498847007751),
 ('explore', 0.5916944742202759),
 ('discuss', 0.591374933719635),
 ('examine', 0.5906602740287781),
 ('deﬁne', 0.5837196111679077),
 ('investigate', 0.5834907293319702),
 ('illustrate', 0.5696419477462769),
 ('outline', 0.5407627820968628),
 ('identify', 0.5361859202384949)]

문장을 전체 돌면서 유사도가 가장 높은 문장을 뽑아서 보여주기

In [ ]:
similarity = [0.0] * len(common_texts)

for i in tqdm_notebook(range(len(common_texts))):
    # 해당 문장의 vector 값 추론
    input_sent = common_texts[i]
    vector = 0
    vector_average_n = 5
    for j in range(vector_average_n):
        vector += model.infer_vector(input_sent)
    vector = vector / vector_average_n
    
    # Search for the most similar sentences
    similar_sentences = model.docvecs.most_similar(positive=[vector], topn=5)
    average = 0.0
    cnt = 0
    for j, simil_point in similar_sentences:
        if common_texts[j] != input_sent:
            average += simil_point
    if cnt:
        average = average / cnt
    
    similarity[i] = average
    
sorted_by_similarity = sorted(enumerate(similarity), key=lambda tup: tup[1], reverse=True)

In [ ]:
for tup in sorted_by_similarity[1000:1100]:
    input_sent = common_texts[tup[0]]
    vector = 0
    vector_average_n = 5
    for i in range(vector_average_n):
        vector += model.infer_vector(input_sent)
    vector = vector / vector_average_n
    
    # Search for the most similar sentences
    similar_sentences = model.docvecs.most_similar(positive=[vector], topn=5)
    
    # Log
    log_path = 'logs/' + model_parameters + '.txt'
    with open(log_path, 'a+', encoding='UTF-8') as f:
        for word in input_sent:
            f.write(word + ' ')
        f.write('<- input_sent\n')
        for pair in similar_sentences:
            sent_num = pair[0]
            similarity = pair[1]
            for word in common_texts[sent_num]:
                f.write(word + ' ')
            f.write('\n')
        f.write('\n')
        
print('done')

랜덤 문장을 선택해 그것과 가장 비슷한 문장을 뽑아주는 것

In [ ]:
import random

num_sents = len(common_texts)  # 문장의 총 개수

for i in range(5):
    randn = random.randrange(num_sents)
    input_sent = common_texts[randn]
    
    vector = 0
    vector_average_n = 100
    for i in range(vector_average_n):
        vector += model.infer_vector(input_sent)
    vector = vector / vector_average_n
        
    # Search for the most similar sentences
    similar_sentences = model.docvecs.most_similar(positive=[vector], topn=5)

    # Log
    log_path = 'logs/' + model_parameters + '.txt'
    with open(log_path, 'a+', encoding='UTF-8') as f:
        for word in input_sent:
            f.write(word + ' ')
        f.write('<- input_sent\n')
        for sent_num, similarity in similar_sentences:
            for word in common_texts[sent_num]:
                f.write(word + ' ')
            f.write('\n')
        f.write('\n')